# Convenient experimenation with Tensorflow

In this notebook we learn about some shortcuts or convenience methods for modelling neuronal networks with tensorflow

Get the necessary data files for the next session from https://www.kaggle.com/datasets/zalando-research/fashionmnist


## Preparing the Fashion MNIST-data

In [ ]:
#!pip install scikit-learn
#!pip install tensorflow==2.10
import pandas as pd
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
print(f"Tesorflow version {tf.__version__}")

#read data with pandas
dataTrain = pd.read_csv('data/fashion-mnist_train.csv')
dataTest = pd.read_csv('data/fashion-mnist_test.csv')

#convert to numpy and initialize variables for training
X_train, y_train = dataTrain.iloc[:, 1:].to_numpy(), dataTrain.iloc[:, 0].to_numpy()
X_test, y_test = dataTest.iloc[:, 1:].to_numpy(), dataTest.iloc[:, 0].to_numpy()

#Define Classnames
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# prepare ground truth as one-hot encoded values
y_one_hot = tf.one_hot(y_train,len(class_names))
y_one_hot_test = tf.one_hot(y_test,len(class_names))


print("Fashion Mnist")
print(f"Training data size: {X_train.shape}")
print(f"Test data size:     {X_test.shape}")
print(f"One Hot encoded:    {y_one_hot.shape}")
print(f"Hardware: {tf.config.list_physical_devices('GPU')}")

# Flatten Layer 

Not always is the data already in the correct format.
So far we always had one-dimensional training samples as a input for NN-Layers. It can also be multidimensional if the next layers support this.

In [ ]:
#Suppose we have the original data not in the original form
X_train_28x28 = X_train.reshape(-1,28,28)
plt.figure(figsize=(1.5, 1.5))
plt.imshow(X_train_28x28[0], cmap="gray_r")


In [ ]:
X_test_28x28 = X_test.reshape(-1,28,28)
plt.figure(figsize=(1.5, 1.5))
plt.imshow(X_test_28x28[0], cmap="gray_r")

In [ ]:
from tensorflow.keras.layers import Flatten

model = Sequential()
model.add(Flatten(input_shape=(28,28)))                                                   # the flatten layer
model.add(Dense(100, activation="sigmoid", input_shape=(784,)))
model.add(Dense(len(class_names), activation="softmax"))
model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(
    X_train_28x28,                                                                        # unflattened input
    y_one_hot,
    epochs=10,
    batch_size=100)

print("Evaluation on Test Data")
print(model.evaluate(X_test_28x28, y_one_hot_test))

# Shortcut: Loss Function SparseCategoricalCrossentropy

Categorial Crossentroypy is a loss function for one-hot-endoded multinomial labels. We can ommit the step of one hot encoding the labels and let the work to the loss-fuction.

$Cost(h_\theta(x),y)$

In [ ]:
model = Sequential()
model.add(Dense(100, activation="sigmoid", input_shape=(784,)))
model.add(Dense(len(class_names), activation="softmax"))
model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=["accuracy"]) # change to sparse_categorial_crossentropy here

model.fit(
    X_train,
    y_one_hot,  # you can dircetly use y_train here
    epochs=10,
    batch_size=100)

print("Evaluation on Test Data")
print(model.evaluate(X_test, y_test)) # the different loss has also effect on evaluate .. we must change the expected labels here


In [ ]:
model.predict(X_test[0:2]) # the loss doesnt change the output here, we still need the argmax

# Testtrain split integrated

In [ ]:
model = Sequential()
model.add(Dense(100, activation="sigmoid", input_shape=(784,)))
model.add(Dense(len(class_names), activation="softmax"))
model.compile(optimizer='sgd', loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(
    X_train,
    y_train,  # used instead of y_one_hot
    epochs=10,
    batch_size=100,
    validation_split=0.3
    #validation_data=(X_test, y_test)
)

print("Evaluation on Test Data")
print(model.evaluate(X_test, y_test)) # the different loss has also effect on evaluate .. we must change the expected labels here


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Creating Sequentical models with the constructor

In [ ]:
model = Sequential([
      Dense(100, activation='sigmoid'),
      Dense(10, activation="softmax")
      ])
model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(
    X_train,
    y_one_hot,
    epochs=10,
    batch_size=100)